In [2]:
import tkinter as tk
from tkinter import ttk, font, simpledialog
import json
import random
import re
import os
import sys

class LabelSeparator(tk.Frame):
    def __init__(self, parent, text="", *args, **kwargs):
        super().__init__(parent, *args, **kwargs)

        # The separator is stretched across the entire width of the frame
        self.separator = ttk.Separator(self, orient=tk.HORIZONTAL)
        self.separator.grid(row=0, column=0, sticky="ew", pady=0)

        # The label is placed above the separator
        self.label = ttk.Label(self, text=text)
        self.label.grid(row=0, column=0)

        # Configure the frame to expand the column, allowing the separator to fill the space
        self.grid_columnconfigure(0, weight=1)

        # Adjust label placement using the 'sticky' parameter to center it
        # 'ns' means north-south, which centers the label vertically in the grid cell
        self.label.grid_configure(sticky="ns")

class ToggleSwitch(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        super().__init__(parent, *args, **kwargs)

        # Initialize variables
        self.is_on = False

        # Define texts for both states
        self.on_text = "Annotator must write annotators rewrite"
        self.off_text = ""

        # Text label
        self.text_label = tk.Label(self, text=self.off_text, font=("Arial", 10))

        # Canvas for the toggle circle
        self.canvas = tk.Canvas(self, width=30, height=21, highlightthickness=0)
        self.circle = self.canvas.create_oval(5, 5, 20, 20, fill="grey")

        # Layout
        self.canvas.grid(row=0, column=0, padx=(0, 10))
        self.text_label.grid(row=0, column=1)

        # Set initial state
        self.update_state()

        # Bind the toggle function to mouse click on the canvas
        #self.canvas.bind("<Button-1>", self.toggle_switch)

    def toggle_switch(self, state, event=None):
        self.is_on = state
        self.update_state()

    def update_state(self):
        color = "green" if self.is_on else "grey"
        text = self.on_text if self.is_on else self.off_text
        self.canvas.itemconfig(self.circle, fill=color)
        self.text_label.config(text=text)

class JsonViewerApp:

    def __init__(self, root, ):
        
        #Main windows settings
        self.root = root
        self.root.title("OneAI ReWrite Annotation Software")     

        if getattr(sys, 'frozen', False):
            application_path = os.path.dirname(sys.executable)

        else:
            application_path = ''

        target_path = os.path.join(application_path, 'target.json')
        
        self.filename = target_path

        # Set the minimum size of the window
        root.minsize(1000, 600)
        root.update()

        self.annotator_id = ''

        #font settings
        self.font_size = 12  # Default font size 

        # Dark and Light mode colors
        self.dark_mode_colors = {
            "background": "#333333", "foreground": "#FFFFFF",
            "button": "#555555", "text": "#FFFFFF", "text_backround": "#444444"
        }

        self.light_mode_colors = {
            "background": "#F7F7F7", "foreground": "#000000",
            "button": "#EFEFEF", "text": "#000000", "text_backround": "#FFFFFF"
        }        
        
        self.pending_changes = {}  # To store changes until saved
        self.rewrite_fields = {}
        
        self.fields_check = False

        self.disable_copy = False
        
        # List of allowed values for score
        self.allowed_score_values =  ['1', '2', '3', '4', '5', '6', '7', '8', '9']
                                         
        # Create a top panel frame
        top_panel_frame = tk.Frame(root)
        top_panel_frame.pack(side=tk.TOP, fill=tk.X)
        
        # Create main PanedWindow
        main_pane = tk.PanedWindow(root, orient=tk.VERTICAL)
        main_pane.pack(fill=tk.BOTH, expand=True)
      
        # Frame for Dialog widgets
        self.dialog_frame_base = tk.Frame(root, height=1)
        main_pane.add(self.dialog_frame_base, stretch="always", height=60)
        LabelSeparator(self.dialog_frame_base, text="Dialog Text").pack(fill=tk.X, side=tk.TOP)

        # tk.Text for dialog
        self.dialog_text = tk.Text(self.dialog_frame_base, wrap=tk.WORD, state='disabled')
        self.dialog_text.pack(fill=tk.BOTH, padx=10, pady=10)  

        # Frame for Rewrite widgets
        self.rewrites_frame_base = tk.Frame(root)
        main_pane.add(self.rewrites_frame_base, stretch="always", height=80)
        LabelSeparator(self.rewrites_frame_base, text="Rewrites").pack(fill=tk.X)


        # Entry for "requires_rewrite"
        self.requires_rewrite_frame = tk.Frame(self.rewrites_frame_base)
        self.requires_rewrite_frame.pack(fill=tk.BOTH, padx=10, pady=10)

        self.requires_rewrite_label = tk.Label(self.requires_rewrite_frame, text="Question Requires Rewrite:")
        self.requires_rewrite_label.grid(row=0, column=0, sticky='w', padx=5, pady=0)
        self.requires_rewrite_entry = tk.Entry(self.requires_rewrite_frame, width = 3)
        self.requires_rewrite_entry.grid(row=0, column=1, sticky='wn', padx=5, pady=0)
        
        def on_key_release(event1=None):
            self.on_entry_edit(self.requires_rewrite_entry, 'requires rewrite', allowed_values=['0', '1'], event=event1)
            self.update_annotator_rewrite_state(event1)

        self.requires_rewrite_entry.bind("<KeyRelease>", on_key_release)
        

        # inside Frame for rewrites annotation entries
        self.rewrite_table_grid = tk.Frame(self.rewrites_frame_base)
        self.rewrite_table_grid.pack(fill=tk.BOTH, padx=10, pady=10)
        
        text = tk.Label(self.rewrite_table_grid, text="Text")
        score = tk.Label(self.rewrite_table_grid, text="Score")
        optimal = tk.Label(self.rewrite_table_grid, text="Optimal")
        
        # Place the labels in the grid
        text.grid(row=0, column=1, sticky='nsew')
        score.grid(row=0, column=2, sticky='nsew')
        optimal.grid(row=0, column=3, sticky='nsew')

        # Configure the frame columns to expand with the window size
        self.rewrite_table_grid.columnconfigure(0, weight = 1)
        self.rewrite_table_grid.columnconfigure(1, weight = 50)
        self.rewrite_table_grid.columnconfigure(2, weight = 1)
        self.rewrite_table_grid.columnconfigure(3, weight = 1)

        #label that appear if there are no rewrites
        self.no_rewrites_label = tk.Label(self.rewrite_table_grid, text="No rewrites in this turn.")
        self.no_rewrites_label.grid(column=1, row=3)
        self.no_rewrites_label.grid_remove()  # This hides the label initially
        
        # Frame for Annotator rewrite widgets
        self.annotator_rewrite_frame_base = tk.Frame(root)
        main_pane.add(self.annotator_rewrite_frame_base, stretch="always", height=30)
        LabelSeparator(self.annotator_rewrite_frame_base, text="Annotator Rewrite").pack(fill=tk.X)
        self.must_annotator_rewrite = ToggleSwitch(self.annotator_rewrite_frame_base)
        self.must_annotator_rewrite.pack(fill=tk.X, padx=10, pady=0)
        
        self.annotator_rewrite_frame_grid = tk.Frame(self.annotator_rewrite_frame_base)
        self.annotator_rewrite_frame_grid.pack(fill=tk.X, padx=10, pady=10)

        
        self.annotator_rewrite_label = tk.Label(self.annotator_rewrite_frame_grid, text="Annotator Rewrite:")
        self.annotator_rewrite_label.grid(row=1, column=0, sticky='w', padx=5, pady=5)
        self.annotator_rewrite_entry = tk.Entry(self.annotator_rewrite_frame_grid)
        self.annotator_rewrite_entry.grid(row=1, column=1, sticky='wne', padx=5, pady=5)

        self.annotator_rewrite_frame_grid.columnconfigure(0,weight=10)
        self.annotator_rewrite_frame_grid.columnconfigure(1,weight=1000)

        self.requires_rewrite_entry.bind("<FocusIn>", self.select_text)
        self.annotator_rewrite_entry.bind("<FocusIn>", self.select_text)

        self.annotator_rewrite_entry.bind("<KeyRelease>", lambda event: self.on_entry_edit(self.annotator_rewrite_entry, 'annotator rewrite'))

        # "<" (Previous) and ">" (Next) buttons next to each other
        self.prev_button = tk.Button(top_panel_frame, text="<", command=self.prev_turn)
        self.prev_button.pack(side=tk.LEFT, padx=(10, 0), pady=10)

        self.next_button = tk.Button(top_panel_frame, text=">", command=self.next_turn)
        self.next_button.pack(side=tk.LEFT)
        
        # "<<" (Previous Dialog) and ">>" (Next Dialog) buttons
        prev_dialog_button = tk.Button(top_panel_frame, text="<<", command=self.prev_dialog)
        prev_dialog_button.pack(side=tk.LEFT, padx=(10, 0), pady=10)

        next_dialog_button = tk.Button(top_panel_frame, text=">>", command=self.next_dialog)
        next_dialog_button.pack(side=tk.LEFT)
        
        # Key bindings for arrow keys
        
        root.bind("<KeyRelease-Down>", self.next_focus)
        root.bind("<KeyRelease-Up>", self.back_focus) 
        if self.disable_copy == True:
            root.event_delete('<<Paste>>', '<Control-v>')
            root.event_delete('<<Copy>>', '<Control-c>')
            pass

        # "Update annotator_id" button on the right side
        update_id_button = tk.Button(top_panel_frame, text="Update Full Name", command=self.update_annotator_id_dialog)
        update_id_button.pack(side=tk.RIGHT, padx=10, pady=10)
 
        # "+" button to increase font size
        increase_font_button = tk.Button(top_panel_frame, text="+", command=self.increase_font_size)
        increase_font_button.pack(side=tk.LEFT, padx=(10, 0), pady=10)

        # "-" button to decrease font size
        decrease_font_button = tk.Button(top_panel_frame, text="-", command=self.decrease_font_size)
        decrease_font_button.pack(side=tk.LEFT, padx=(0, 10), pady=10)
        
        # Dark Mode Button
        self.dark_mode_button = tk.Button(top_panel_frame, text="Dark Mode", command=self.toggle_dark_mode)
        self.dark_mode_button.pack(side=tk.LEFT, padx=10, pady=10)

        # Dark mode state
        self.dark_mode = True

        # Save Button at the bottom
        self.save_button = tk.Button(root, text="Save and Next", command=self.next_turn)
        self.save_button.pack(side=tk.BOTTOM, pady=10)
        self.save_button.bind("<Return>", self.next_turn)
        
        # Current dialog and turn labels
        self.current_dialog_label = tk.Label(top_panel_frame, text="")
        self.current_dialog_label.pack(side=tk.LEFT, padx=10, pady=10)

        self.current_turn_label = tk.Label(top_panel_frame, text="")
        self.current_turn_label.pack(side=tk.LEFT, padx=10, pady=10)


        # Load JSON and display data
        self.start_at = 1 #set the first turn to start annotate
        self.json_data = self.load_json("target.json")
        self.current_dialog_index = 0
        self.current_turn_index = self.start_at
        self.focus_list = []
        self.focus_index = 0
        if self.check_and_update_annotator_id(start=True):
        
            # After loading JSON data, find the first turn with an empty score
            self.find_next_unscored_turn()
            self.display_dialog()
            self.toggle_dark_mode()
            self.increase_font_size()
            self.focused_element = None

    def select_text(self, event=None):
            event.widget.select_range(0,tk.END)
        
    def find_next_unscored_turn(self):
        for dialog_index, dialog_id in enumerate(self.json_data):
            dialog_data = self.json_data[dialog_id]
            turn_index = dialog_data['number_of_turns']
            turn_id = str(turn_index)
            turn_data = dialog_data[turn_id]
            for rewrite_key, rewrite_value in turn_data.items():
                if isinstance(rewrite_value, dict) and 'score' in rewrite_value.keys() and 'optimal' in rewrite_value.keys():
                    score = rewrite_value.get('score')
                    if not score:
                        # Set the current dialog and turn index
                        self.current_dialog_index = dialog_index
                        self.current_turn_index = turn_index
                        return  # Exit after finding the first unscored turn

    def update_current_turn_dialog_labels(self):
        # Update labels with current dialog and turn information
        dialog_number = self.current_dialog_index + 1
        turn_number = self.current_turn_index + 1
        total_dialogs = len(self.json_data)
        total_turns = self.current_dialog_turn_number

        self.current_dialog_label.config(text=f"Dialog: {dialog_number}/{total_dialogs}")
        self.current_turn_label.config(text=f"Turn: {turn_number-self.start_at}/{total_turns-self.start_at}")
        
    def are_all_fields_filled(self):
        missing_fields = []

        # Check if annotator_id is filled
        first_dialog_id = next(iter(self.json_data))  
        if not self.json_data[first_dialog_id].get('annotator_id'):
            missing_fields.append("annotator name")

        # Check if annotator rewrite is filled
        dialog_id = self.current_dialog_id()
        turn_id = self.current_turn_id()
        
        if self.json_data[dialog_id][turn_id].get('requires rewrite') is None or self.requires_rewrite_entry.get() == '':
            missing_fields.append("Requires Rewrite")
        
        if self.must_annotator_rewrite.is_on == True:
            if self.json_data[dialog_id][turn_id].get('annotator rewrite') is None:
                missing_fields.append("Annotator Rewrite")

        # Check if all rewrites have scores
        rewrite_name_index = 1
        for rewrite_key, row in self.rewrite_fields.items():
            score = row['score'].get()
            optimal = row['optimal'].get()
            if score == '':
                missing_fields.append(f"Score for 'Rewrite {rewrite_name_index}'")
            if optimal == '':
                missing_fields.append(f"Optimal for 'Rewrite {rewrite_name_index}'")
            rewrite_name_index+=1
            
            

        if missing_fields and self.fields_check:
            return False, "The following fields are missing: " + ", ".join(missing_fields) + ". Please fill them in before proceeding."
        return self.annotator_rewrite_unique()
    
    def normalize_rewrite(self, rewrite):
         # Convert the text to lowercase
        lower_text = rewrite.lower()

        # Remove anything that's not a lowercase character
        cleaned_text = ''.join(char for char in lower_text if char.islower())

        return cleaned_text
    
    def check_similarity(self, rewrite1, rewrite2):
        if rewrite1 == rewrite2:
            return True

    def annotator_rewrite_unique(self):
        if self.requires_rewrite_entry.get() == '0':
            return True, ""
            
        annotator_rewrite_text = self.normalize_rewrite(self.annotator_rewrite_entry.get())

        # Fetch the dialog and turn IDs
        dialog_id = self.current_dialog_id()
        
        # Fetch the Question from the dialog data
        question = ''
        
        for dialog in self.json_data[dialog_id]['dialog']:
            if dialog['turn_num'] == self.current_turn_index + 1: 
                question = self.normalize_rewrite(dialog['original_question'])
                break

        # Check if annotator rewrite is identical to original question
        if self.check_similarity(annotator_rewrite_text , question):
            return False, "Annotator Rewrite identical to the original question"
        
        # Check against other rewrites
        rnum = 1
        for row in self.rewrite_fields.values():
            the_text = row['text'].get("1.0", tk.END)
            the_text = self.normalize_rewrite(the_text)
            if self.check_similarity(annotator_rewrite_text , the_text):
                return False, f"Annotator Rewrite identical to Rewrite {rnum}"
            rnum += 1
            
        return True, ""
        
    def update_json(self):
        if self.annotator_rewrite_entry.get() == '':
            self.json_data[self.current_dialog_id()][self.current_turn_id()]['annotator rewrite'] = -1
            
        # Save current state and move to next if all fields are filled
        self.save_json()
        
    def json_update_rewrite_data(self, rewrite_key, new_score, field, prnt=False):
        
        dialog_id = self.current_dialog_id()
        turn_id = self.current_turn_id()

        if new_score is None:
            self.json_data[dialog_id][turn_id][rewrite_key][field] = None
            return 
      
        if field == "score":  # "#2" is the 'Score' column

            if rewrite_key in self.json_data[dialog_id][turn_id]:
                self.json_data[dialog_id][turn_id][rewrite_key]['score'] = int(new_score)
                self.update_identical_rewrites(dialog_id, turn_id, rewrite_key, int(new_score), 'score')
                if prnt: print(f'updated rewrite: {rewrite_key} column: {field} with {new_score}')
            else:
                if prnt: print(f"Rewrite with key '{rewrite_key}' not found.")

        
        elif field == "optimal":  # "#3" is the 'Optimal' column
            if rewrite_key in self.json_data[dialog_id][turn_id]:
                self.json_data[dialog_id][turn_id][rewrite_key]['optimal'] = int(new_score)
                self.update_identical_rewrites(dialog_id, turn_id, rewrite_key, int(new_score), 'optimal')
                if prnt: print(f'updated rewrite: {rewrite_key} column: {field} with {new_score}')
            else:
                if prnt: print(f"Rewrite with key '{rewrite_key}' not found.")
                
        else:
            if prnt: print('column not found')

    def json_is_valid(self, rewrite_key, row, field, event1=None):

        new_value = row[field].get()

        # Check if the new value is in the list of allowed values or if allowed values are None
        if new_value == '':
            return False #should've changed the json to null here fuck
            
        else:
            #If Score column selected
            if field == 'score':
                if self.allowed_score_values is None or new_value in self.allowed_score_values:
                    #change optimal if nesseceray
                    if int(new_value) != self.json_data[self.current_dialog_id()][self.current_turn_id()][rewrite_key]['score']:
                        for temp_key, temp_row in self.rewrite_fields.items():
                            temp_row['optimal'].delete(0,tk.END)     
                            self.json_update_rewrite_data(temp_key, None, 'optimal')

                    self.json_update_rewrite_data(rewrite_key, row['score'].get(), 'score')

                    return True
                else:
                    # Show a warning message for invalid input
                    row['score'].delete(0,tk.END)
                    allowed_values_str = ', '.join(self.allowed_score_values) if self.allowed_score_values is not None else 'any value'
                    tk.messagebox.showwarning("Invalid Score", f"Invalid score '{str(new_value)}'. Allowed values are: {allowed_values_str}")
                    return False
                    

            #If Optimal column selected
            if field == 'optimal':
                if not self.allow_optimal(event=event1):
                    row['optimal'].delete(0,tk.END)
                    return
                
                if new_value in ['0', '1']: 
                    #check if 1 is valid:
                    if new_value == '1':
                        if not self.optimal_valid(rewrite_key, new_value):
                            row['optimal'].delete(0,tk.END)
                            tk.messagebox.showwarning("Invalid Score", f"Score is not the highest")  
                            return False
                    self.json_update_rewrite_data(rewrite_key, row['optimal'].get(), 'optimal')
                    self.match_optimal(rewrite_key, new_value)

                    self.toggle_greenie()
                    return True
                else:
                    row['optimal'].delete(0,tk.END)
                    tk.messagebox.showwarning("Invalid Score", f"Invalid input '{str(new_value)}'. Allowed values are: 0 or 1")
                    return False
                
        return None

    def toggle_greenie(self):
         #check if optimal rewrite exists
        all_filled, optimal_exits = self.all_optimal_filled()

        if all_filled:
            if optimal_exits:
                self.must_annotator_rewrite.toggle_switch(False)
            elif self.requires_rewrite_entry.get() != '0':
                self.must_annotator_rewrite.toggle_switch(True)
        else:
            self.must_annotator_rewrite.toggle_switch(False)
    
    def all_optimal_filled(self):
        optimal_exists = False


        rewrite_name_index = 1
        for row in self.rewrite_fields.values():
            optimal = row['optimal'].get()
            if optimal == '1':
                optimal_exists = True
            if optimal == '':
                return False, optimal_exists
            rewrite_name_index+=1
        return True, optimal_exists
    

    def match_optimal(self, rewrite_key, optimal):
        score = self.rewrite_score(rewrite_key)
        for temp_key, row in self.rewrite_fields.items():
            temp_score = self.rewrite_score(temp_key)
            if rewrite_key != temp_key and temp_score == score:
                row['optimal'].delete(0,tk.END)
                row['optimal'].insert(0, optimal)
                self.json_update_rewrite_data(temp_key, int(optimal), 'optimal')
    
    def toggle_dark_mode(self):
        self.dark_mode = not self.dark_mode
        self.apply_dark_mode(self.root)

        # Update the button text
        self.dark_mode_button.config(text="Light Mode" if self.dark_mode else "Dark Mode")

    def apply_dark_mode(self, widget):
        
        colors = self.dark_mode_colors if self.dark_mode else self.light_mode_colors


        # Apply styles to the widget
        if isinstance(widget, (tk.Text, tk.Entry)):
            widget.config(bg=colors["text_backround"], fg=colors["foreground"])
        elif isinstance(widget, tk.Label) or isinstance(widget, tk.Button):
            widget.config(bg=colors["background"], fg=colors["foreground"])      
        elif isinstance(widget, tk.Canvas):
            widget.config(bg=colors["background"])
        elif isinstance(widget, ttk.Widget):
            # Skip other ttk widgets as they are handled by the style configuration
            pass
        else:
            # For other widgets, try setting the background
            try:
                widget.config(bg=colors["background"])
            except tk.TclError:
                pass  # Ignore widgets that do not support background configuration

        # Recursively apply to all children
        for child in widget.winfo_children():
            self.apply_dark_mode(child)
        
    def increase_font_size(self):
        if self.font_size < 30:
            self.font_size += 1
            self.update_font_size(self.root)
            self.update_window_size(enlarge=True)

    def decrease_font_size(self):
        if self.font_size > 10:
            self.font_size -= 1
            self.update_font_size(self.root)
            self.update_window_size(enlarge=False)
            
    def update_font_size(self, widget):
        new_font = font.Font(size=self.font_size)

        try:
            widget.configure(font=new_font)
        except:
            pass

        for child in widget.winfo_children():
            self.update_font_size(child)

    def update_window_size(self, enlarge):
        if enlarge:
            num = 40
        else:
            num = -40

        # Get the current size of the window
        current_width = self.root.winfo_width()
        current_height = self.root.winfo_height()

        # Calculate a new height, but ensure it's within the screen's limits
        screen_height = self.root.winfo_screenheight()
        new_height = min(current_height + num, screen_height)

        # Calculate a new Width, but ensure it's within the screen's limits
        screen_width = self.root.winfo_screenwidth()
        new_width = min(current_width + num*2, screen_width)

        # Update the window size using geometry
        self.root.geometry(f"{new_width}x{new_height}")
        self.root.update()

    def check_and_update_annotator_id(self, start=False):
        first_dialog_id = next(iter(self.json_data))  # Get the first key in the JSON data
        if 'annotator_id' not in self.json_data[first_dialog_id] or self.json_data[first_dialog_id]['annotator_id'] is None:
            return self.update_annotator_id_dialog(start1 = start) 
        return True
    
    def update_annotator_id_dialog(self, start1=False):
        first_dialog_id = next(iter(self.json_data))  # Get the first key in the JSON data
        current_id = self.json_data[first_dialog_id].get('annotator_id')  # Get current annotator_id

        new_id = simpledialog.askstring("Input", "Enter annotator's *full* name:", initialvalue=current_id, parent=self.root)

        def contains_english_letter(s):
            if s is None:
                return False
            return bool(re.search('[a-zA-Z]', s))
        
        if contains_english_letter(new_id):
            self.annotator_id = new_id
            self.update_annotator_id()
            return True

        elif start1 == True:
            self.root.destroy()
            return False
        
        else:
            return 
             
    def update_annotator_id(self):
        for dialog_id in self.json_data:
            self.json_data[dialog_id]['annotator_id'] = self.annotator_id
        self.save_json()
    
    def update_dialog_text(self, new_text):
        # Enable the widget to modify text
        self.dialog_text.config(state='normal')

        # Update the text
        self.dialog_text.delete(1.0, tk.END)
        self.dialog_text.insert(tk.END, new_text)

        # Disable the widget to prevent user edits
        self.dialog_text.config(state='disabled')
          
    def load_json(self, filename):
        data = None
        try:
            with open(self.filename, 'r') as file:
                data = json.load(file)

        except FileNotFoundError:
            tk.messagebox.showerror("Annotation File Not Found", f"Please place target.json inside the same folder with the program")
            self.root.destroy()
            

        # Shuffling and storing rewrites
        self.shuffled_rewrites = {}
        self.identical_rewrites = {}
        for dialog_id, dialog_data in data.items():
            
            for turn_id, turn_data in dialog_data.items():
                # Check if turn_id is a digit
                if turn_id.isdigit():

                    # Select keys that contain 'rewrite' and do not contain 'annotator'
                    rewrites = []
                    
                    for key, value in turn_data.items():
                        if isinstance(value, dict) and 'score' in value.keys() and 'optimal' in value.keys():
                            exist = False
                            for rewrite in rewrites:
                                if value['text'] == rewrite[1]['text']:
                                    self.identical_rewrites[(dialog_id, turn_id, rewrite[0])].append(key)
                                    exist = True
                                    
                            if not exist:
                                rewrites.append((key,value))
                                self.identical_rewrites[(dialog_id, turn_id, key)] = []
                            
                    
                    random.shuffle(rewrites)
                    self.shuffled_rewrites[(dialog_id, turn_id)] = rewrites
        
        
        return data
     
    def allow_optimal(self, event=None):
        if event.keysym == "Up" or event.keysym == "Down": 
            return True
        #Checks if all the scores are filled
        else:
            for row in self.rewrite_fields.values():
                if row['score'].get() == '':
                    tk.messagebox.showwarning('Score not filled', 'Please fill all scores before choosing optimal rewrites')
                    self.root.after(1, self.root.focus_set)
                    row['optimal'].delete(0,tk.END)
                    return False  
                else:
                    return True
            
    def optimal_valid(self, rewrite_key, optimal):
        if optimal == '1':
            for temp_key, temp_value in self.rewrite_fields.items():
                if temp_key != rewrite_key and temp_value['score'].get() > self.rewrite_score(rewrite_key):
                    return False
                
        return True
    
    def rewrite_score(self, rewrite_key):     
        score = self.rewrite_fields[rewrite_key]['score'].get()
        return score
    
    def rewrite_optimal(self, rewrite_key):    
        optimal = self.rewrite_fields[rewrite_key]['optimal'].get()
        return optimal
             
    def update_identical_rewrites(self, dialog_id, turn_id, rewrite, new_score, field):
        """
        If a Rewrite has Identical Rewrites not shown in the program, update them with identical values
        """
        
        identical_rewrite = self.identical_rewrites.get((dialog_id, turn_id, rewrite))
        
        
        for t_rewrite in identical_rewrite:
            self.json_data[dialog_id][turn_id][t_rewrite][field] = int(new_score)
        
    def save_json(self):
        # Write the updated json_data back to the file
        with open(self.filename, 'w') as file:
            json.dump(self.json_data, file, indent=4)
            
    def current_dialog_id(self):
        return list(self.json_data.keys())[self.current_dialog_index]

    def current_turn_id(self):
        return str(self.current_turn_index)
    
    def display_dialog(self):
        dialog_text_content = ""

        # Construct the dialog text content
        dialog_id = self.current_dialog_id()
        dialog_data = self.json_data[dialog_id]
        self.current_dialog_turn_number = dialog_data['number_of_turns']

        for dialog in dialog_data['dialog']:
            if dialog['turn_num'] <= self.current_turn_index + 1:
                # Format each turn
                turn_text = f"Turn {dialog['turn_num'] - 1}:\n"
                turn_text += f"Q: {dialog['original_question']}\n"
                if dialog['turn_num'] != self.current_turn_index + 1:
                    turn_text += f"A: {dialog['answer']}\n"
                turn_text += "-" * 40 + "\n"  # S1eparator line

                # Append this turn's text to the dialog text content
                dialog_text_content += turn_text
                
        self.update_current_turn_dialog_labels()

        # Update the dialog text widget using the new method
        self.update_dialog_text(dialog_text_content)

        # Scroll to the end of the dialog text
        self.dialog_text.see(tk.END)

        # Display the current turn
        self.display_turn()
             
    def rewrite_identical_to_current_question(self, rewrite_key, rewrite_text):
        # Fetch the dialog and turn IDs
        dialog_id = self.current_dialog_id()
        turn_id = self.current_turn_id()
        
        # Fetch the Question from the dialog data
        question = ''
        
        for dialog in self.json_data[dialog_id]['dialog']:
            if dialog['turn_num'] == self.current_turn_index + 1: 
                question = dialog['original_question']
                
        if question == rewrite_text:
            
            self.json_update_rewrite_data(rewrite_key, -1, 'score')
            self.json_update_rewrite_data(rewrite_key, -1, 'optimal') 
            return True
        
        return False
             
    def display_turn(self):        
        # Clear existing content
        self.reset_rows(self.rewrite_fields)
        
        

        # Fetch the dialog and turn IDs
        dialog_id = self.current_dialog_id()
        turn_id = self.current_turn_id()

        # Use the stored shuffled rewrites
        rewrites = self.shuffled_rewrites.get((dialog_id, turn_id), [])
        self.rewrite_fields = {}
        row = 1
        valid_rewrites_len = 0


        
        # Display the rewrites in the program
        for rewrite_key, rewrite_value in rewrites:
            
            if self.rewrite_identical_to_current_question(rewrite_key, rewrite_value['text']):
                
                continue
        
            # Check if score is None or not a string, and replace it with an empty string
            score = rewrite_value.get('score', '')
            if score is None or not isinstance(score, int):
                score = ''
                
            optimal = rewrite_value.get('optimal', '')
            if optimal is None or not isinstance(optimal, int):
                optimal = ''

            
            self.rewrite_fields[rewrite_key] = self.add_row(row, rewrite_key)
            row += 1
            
            self.rewrite_fields[rewrite_key]['text'].insert(tk.END, rewrite_value.get('text', ''))
            self.rewrite_fields[rewrite_key]['score'].insert(0, score)
            self.rewrite_fields[rewrite_key]['optimal'].insert(0, optimal)
            self.rewrite_fields[rewrite_key]['text'].config(state = 'disabled' )
            valid_rewrites_len += 1

        if valid_rewrites_len == 0:
            self.no_rewrites_label.grid()  # Show the label
            
        else:
            self.no_rewrites_label.grid_remove()  # Hide the label

                

        # Update annotator rewrite text
        self.update_entry_text(self.annotator_rewrite_entry, 'annotator rewrite')
        self.update_entry_text(self.requires_rewrite_entry, 'requires rewrite')
        self.update_annotator_rewrite_state()
        self.create_focus_list()
        self.toggle_greenie()
        #self.test()

    def widget_to_rewrite_key(self, widget):
        for key, row in self.rewrite_fields.items():
            for field in row.values():
                if field == widget:
                    return key
                
    def clear_optimal_focus_out(self, event=None):
        rewrite_key = self.widget_to_rewrite_key(event.widget)
        if self.rewrite_fields[rewrite_key]['score'].get() == '':
            self.rewrite_fields[rewrite_key]['optimal'].delete(0,tk.END) 
           
    def add_row(self, rnum, rewrite_key):
        rewrite_label = tk.Label(self.rewrite_table_grid, text=f"Rewrite {rnum}")
        if self.dark_mode:
                rewrite_label.config(bg=self.dark_mode_colors["background"], fg=self.dark_mode_colors["text"]) 
        rewrite_label.grid(column=0,row=rnum)
        text = tk.Text(self.rewrite_table_grid, height=1, wrap='none')
        score = tk.Entry(self.rewrite_table_grid, width=5)
        optimal = tk.Entry(self.rewrite_table_grid, width=5)

        fields_dict = {'text': text, 'score': score, 'optimal': optimal}   

        optimal.bind("<Button-1>", self.allow_optimal)
        optimal.bind("<KeyRelease>", self.allow_optimal)
        optimal.bind("<KeyRelease>", lambda event: self.json_is_valid(rewrite_key, fields_dict, field='optimal', event1=event))
        score.bind("<KeyRelease>", lambda event: self.json_is_valid(rewrite_key, fields_dict, field='score', event1=event))
        score.bind("<FocusOut>", self.clear_optimal_focus_out)

        score.bind("<FocusIn>", self.select_text)
        optimal.bind("<FocusIn>", self.select_text)
        
        
        cnum = 1
        for field in fields_dict.values():
            if self.dark_mode:
                field.config(bg=self.dark_mode_colors["text_backround"], fg=self.dark_mode_colors["foreground"])
            field.grid(row=rnum, column=cnum, sticky='we', padx=5, pady=5)
            cnum += 1
            
        return fields_dict
    
    def reset_rows(self, rows):
        for row in rows.values():
            for entry in row.values():
                entry.destroy()
        
        for widget in self.rewrite_table_grid.grid_slaves(column=0):
            widget.destroy()

    def update_entry_text(self, entry, key_id):
        """
        Updates the Annotator Rewrite or requires_rewrite
        """
        entry.config(state='normal')
        dialog_id = self.current_dialog_id()
        turn_id = self.current_turn_id()
        
        # Clear the Entry widget
        entry.delete(0, tk.END)

        # Fetch and insert text into the Entry widget
        entry_text = self.json_data[dialog_id][turn_id].get(key_id, '')
  

        if entry_text is not None and entry_text != -1:
            entry.insert(0, entry_text)
        
        # Select all text in the Entry widget
        entry.select_range(0, tk.END)   
        
    def update_annotator_rewrite_state(self, event=None):
        """
        Depending on the requires_rewrite input, changes the state of the annotator_rewrite text
        """
        
        dialog_id = self.current_dialog_id()
        turn_id = self.current_turn_id()
        
        if (self.requires_rewrite_entry.get().strip() == '1'):
            self.annotator_rewrite_entry.config(state='normal')
            if self.annotator_rewrite_entry not in self.focus_list:
                self.focus_list.insert(-1, self.annotator_rewrite_entry)
            if self.json_data[dialog_id][turn_id]['annotator rewrite'] == -1:
                self.json_data[dialog_id][turn_id]['annotator rewrite'] = None
            
        elif (self.requires_rewrite_entry.get().strip() == '0'):
            #self.annotator_rewrite_entry.delete(0,tk.END)
            #self.annotator_rewrite_entry.config(state='disabled')
            #if self.annotator_rewrite_entry in self.focus_list:
            #    self.focus_list.remove(self.annotator_rewrite_entry)
            # Update the json_data with -1
            #self.json_data[dialog_id][turn_id]['annotator rewrite'] = -1
            pass
             
    def on_entry_edit(self, entry, key_id, event=None, allowed_values = None):
        """
        Edits Annotator Rewrite or requires_rewrite
        """
        # Retrieve text from an Entry widget
        updated_text = entry.get()
        dialog_id = self.current_dialog_id()
        turn_id = self.current_turn_id()

        if updated_text == '':
            self.json_data[dialog_id][turn_id][key_id] = None
            return False

        
        if allowed_values is None or updated_text in allowed_values:

            # Update the json_data with the new text
            if allowed_values is not None:
                self.json_data[dialog_id][turn_id][key_id] = int(updated_text)
            else:
                self.json_data[dialog_id][turn_id][key_id] = updated_text
            return True

        else:
            entry.delete(0, tk.END)
            tk.messagebox.showwarning("Invalid Score", f"Invalid input '{str(updated_text)}'. Allowed values are: 0 or 1")
            return False

    def prev_turn(self):
        
        self.update_json()
        if self.current_turn_index > self.start_at:
            self.current_turn_index -= 1
        else:
            if self.current_dialog_index > 0:
                self.current_dialog_index -= 1
                dialog_id = self.current_dialog_id()
                dialog_data = self.json_data[dialog_id]
                self.current_turn_index = dialog_data['number_of_turns'] - 1
            else:
                return  # Do nothing if already at the first dialog and turn
            
        self.update_current_turn_dialog_labels()

        self.display_dialog()

        self.update_font_size(self.root)

    def next_turn(self, event=None):
       
        are_filled, message = self.are_all_fields_filled()
        if not are_filled:
            tk.messagebox.showwarning("Warning", message)
            return
        
        self.update_json()

        # Proceed to next turn if all fields are filled
        if self.current_turn_index < self.current_dialog_turn_number - 1:
            self.current_turn_index += 1
        else:
            if self.current_dialog_index < len(self.json_data) - 1:
                self.current_dialog_index += 1
                self.current_turn_index = self.start_at
                print(f"dialog= {self.current_dialog_id()}")
            else:
                print("finished annotation file")
                return  # Do nothing if already at the last dialog and turn
            
        self.update_current_turn_dialog_labels()

        self.display_dialog()

        self.update_font_size(self.root)
        
    def prev_dialog(self):
       
        if self.current_dialog_index > 0:
                self.update_json()
                self.current_dialog_index -= 1
                self.current_turn_index = self.start_at
                self.display_dialog()
                self.update_font_size(self.root)
        else:
            tk.messagebox.showwarning("Warning", "This is the first dialog")

    def next_dialog(self):
      
        if self.current_dialog_index < len(self.json_data) - 1:
            if self.fields_check:
                if self.are_all_turns_filled(self.current_dialog_index):
                    self.update_json()
                    self.current_dialog_index += 1
                    self.current_turn_index = self.start_at
                    self.display_dialog()
                    self.update_font_size(self.root)
                else:
                    tk.messagebox.showwarning("Warning", "Not all turns in this dialog are filled")
            else:
                self.update_json()
                self.current_dialog_index += 1
                self.current_turn_index = self.start_at
                self.display_dialog()
                self.update_font_size(self.root)

    def are_all_turns_filled(self, dialog_index):
        dialog_id = list(self.json_data.keys())[dialog_index]
        dialog_data = self.json_data[dialog_id]
        for turn_id in range(1+self.start_at, dialog_data['number_of_turns']+1):
            turn_data = dialog_data[str(turn_id)]
            for rewrite_key, rewrite_value in turn_data.items():
                if isinstance(rewrite_value, dict) and 'score' in rewrite_value.keys() and 'optimal' in rewrite_value.keys():
                    bl, string = self.are_all_fields_filled()
                    if not bl:
                        return False
        return True
    
    
     #add another option for showing the rewrites
        
    def create_focus_list(self):
            self.focus_list = []
            self.focus_index = 0
            self.first_focus_action = True
            self.focus_list.append(self.requires_rewrite_entry)
            for row in self.rewrite_fields.values():
                self.focus_list.append(row['score'])
            for row in self.rewrite_fields.values():
                self.focus_list.append(row['optimal'])
            
            self.focus_list.append(self.annotator_rewrite_entry)
            self.focus_list.append(self.save_button)
    
    def next_focus(self, event=None):
        if self.first_focus_action:
            self.first_focus_action = False
        else:
            self.focus_index += 1

        if self.focus_index >= len(self.focus_list):
            self.focus_index = 0

        self.focus_list[self.focus_index].focus()

    def back_focus(self, event=None):
        if self.first_focus_action:
            self.focus_index = len(self.focus_list)  # Start from the end
            self.first_focus_action = False

        self.focus_index -= 1
        if self.focus_index < 0:
            self.focus_index = len(self.focus_list) - 1

        self.focus_list[self.focus_index].focus()


def main():
    if getattr(sys, 'frozen', False):
        application_path = os.path.dirname(sys.executable)

    else:
        application_path = ''

    target_path = os.path.join(application_path, 'target.json')

    # Check if the file exists
    if not os.path.isfile(target_path):
        tk.messagebox.showerror("Annotation File Not Found", "Please place target.json inside the same folder with the program")

        return


    root = tk.Tk()
    app = JsonViewerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

